In [1]:
from model_builder import model_builder
from data_gen import simulate_private_equity_cashflows
import numpy as np

In [15]:
cashflows = simulate_private_equity_cashflows()

In [16]:
# Normalize the cashflows of each fund in the market

def fund_weights(cashflows):
    funds = cashflows['FundID'].unique() 
    weights = {}
    for fund in funds:
        fund_cashflows = cashflows[cashflows['FundID'] == fund]
        weights[fund] = -fund_cashflows['Cashflow'].sum()
    return weights

cashflows_contr = cashflows[cashflows['Cashflow'] < 0]
eq_cashflows_contr = fund_weights(cashflows_contr)

def eq_calc(row):
    fund = row['FundID']
    cf = row['Cashflow']
    
    return cf / eq_cashflows_contr[fund]

# Apply the function row-wise
cashflows['eq_cashflow'] = cashflows.apply(eq_calc, axis=1)

# Drop and rename
cashflows = cashflows.drop(columns=['Cashflow']).rename(columns={'eq_cashflow': 'Cashflow'})

cashflows

,FundID,VintageYear,Strategy,Geography,FundQuality,Quarter,date,Cashflow
0,0,1992,Buyout,Rest of World,Good,0,1992-03-31,-0.143824
1,0,1992,Buyout,Rest of World,Good,1,1992-06-30,-0.092189
2,0,1992,Buyout,Rest of World,Good,2,1992-09-30,-0.084082
3,0,1992,Buyout,Rest of World,Good,3,1992-12-31,0.000000
4,0,1992,Buyout,Rest of World,Good,4,1993-03-31,-0.073827
...,...,...,...,...,...,...,...,...
118681,2399,2001,Venture,North America,Good,48,2013-03-31,0.061353
118682,2399,2001,Venture,North America,Good,49,2013-06-30,0.000000
118683,2399,2001,Venture,North America,Good,50,2013-09-30,0.000000
118684,2399,2001,Venture,North America,Good,51,2013-12-31,0.000000


In [17]:
def portfolio_cashflows(cashflows, portfolio_size):
    # Randomly select fund IDs
    selected_funds = np.random.choice(cashflows['FundID'].unique(), size=portfolio_size, replace=False)
    
    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in selected_funds}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(selected_funds)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [10]:
port_cfs = portfolio_cashflows(cashflows, 200)
port_cfs = port_cfs.copy()
model = model_builder(cashflows, port_cfs)

### Monte-carlo simulation

In [18]:
market_model = model_builder(cashflows, cashflows)
market_model

({'Portfolio MOIC': 1.4055335034178185,
  'Market MOIC': 1.4055335034178185,
  'Timing Alpha MOIC': -2.220446049250313e-16,
  'Strategy Alpha MOIC': -2.220446049250313e-16,
  'Geography Alpha MOIC': -2.220446049250313e-16,
  'Sizing Alpha MOIC': 0.0,
  'Residual Alpha MOIC': 6.661338147750939e-16},
 {'Portfolio IRR': 0.0564498673454297,
  'Market IRR': 0.0564498673454297,
  'Timing Alpha IRR': 2.0816681711721685e-17,
  'Strategy Alpha IRR': 5.551115123125783e-17,
  'Geography Alpha IRR': 2.0816681711721685e-17,
  'Sizing Alpha IRR': -1.3877787807814457e-17,
  'Residual Alpha IRR': -8.326672684688674e-17})

In [24]:
number_of_simulations = 50
fund_size = 200
moic_results = []
irr_results = []

for i in range(number_of_simulations):
    port_cfs = portfolio_cashflows(cashflows, fund_size)
    port_cfs = port_cfs.copy()
    model = model_builder(cashflows, port_cfs)
    moic_results.append(model[0])
    irr_results.append(model[1])


In [25]:
def average_dicts(dict_list):
    """Compute the average of a list of dictionaries."""
    keys = dict_list[0].keys()
    avg_dict = {}
    for key in keys:
        avg_dict[key] = sum(d[key] for d in dict_list) / len(dict_list)
    return avg_dict

In [26]:
avg_moic_decomposition = average_dicts(moic_results)
avg_irr_decomposition = average_dicts(irr_results)

In [27]:
avg_irr_decomposition

{'Portfolio IRR': 0.05586047065361904,
 'Market IRR': 0.05644986734542974,
 'Timing Alpha IRR': -3.838962951460467e-05,
 'Strategy Alpha IRR': -7.542901584329408e-05,
 'Geography Alpha IRR': -3.1714074922789965e-05,
 'Sizing Alpha IRR': -0.00036070214041277333,
 'Residual Alpha IRR': -8.316183111719096e-05}

In [28]:
avg_moic_decomposition

{'Portfolio MOIC': 1.401314540995462,
 'Market MOIC': 1.4055335034178191,
 'Timing Alpha MOIC': -0.00017208641137145709,
 'Strategy Alpha MOIC': -0.00039349873434086025,
 'Geography Alpha MOIC': -0.0003161807849346232,
 'Sizing Alpha MOIC': -0.002714362627578342,
 'Residual Alpha MOIC': -0.0006228338641310404}